In [41]:
# ----------
# User Instructions:
# 
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, your function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, your function should return the string
# 'fail'
# ----------

# Grid format:
#   0 = Navigable space
#   1 = Occupied space

import numpy as np


grid = np.array([[0, 0, 1, 0, 0, 0],
                [0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 1, 0],
                [0, 0, 1, 1, 1, 0],
                [0, 0, 0, 0, 1, 0]])
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

In [64]:
def add_2tuple(tuple0, tuple1):
    return (tuple0[0] + tuple1[0], tuple0[1] + tuple1[1])

class Map:
    
    def __init__(self, grid, start, end, cost):
        self.grid = grid
        self.start = tuple(start)
        self.end = tuple(end)
        self.cost = cost
        self.dims = (len(grid), len(grid[0]))

        self.distances = np.full_like(self.grid, -1)
        self.distances[self.start] = 0

        self.move_types = [(1,0), (0,1), (-1,0), (0,-1)]
        self.move_tuple_2_string = {
            ( 1, 0): 'v',
            ( 0, 1): '>',
            (-1, 0): '^',
            ( 0,-1): '<',
        }
        self.move_string_2_tuple = {
            'v': ( 1, 0),
            '>': ( 0, 1),
            '^': (-1, 0),
            '<': ( 0,-1),
        }
        self.moves_to_get_to_loc = np.full(self.dims, ' ')
        
        self.locs_to_update = [(self.city_block_heuristic(0, self.start), self.start)]
        
        self.step = 0
        self.expand = np.full_like(self.grid, -1)

        
    def valid_and_legal_move(self, loc):
        x,y = loc
        if 0 <= x < self.dims[0] and 0<= y < self.dims[1]:
            return bool(1 - self.grid[x, y])
        else:
            return False
    
    
    def city_block_heuristic(self, d, loc):
        return d + (self.dims[0] - 1 - loc[0]) + (self.dims[1] - 1 - loc[1])
 

    def expand_loc(self, loc):
        d = self.distances[loc]
        # go through moves and look at child nodes, computing heuristic and adding to locs_to_update
        for move in self.move_types:
            next_loc = add_2tuple(loc, move)
            
            if self.valid_and_legal_move(next_loc) and self.distances[next_loc] == -1:
                self.distances[next_loc] = d + cost

                heuristic = self.city_block_heuristic(d + cost, next_loc)
                self.locs_to_update.append((heuristic, next_loc))
                self.locs_to_update.sort()

                # add the move to the move list
                self.moves_to_get_to_loc[next_loc] = self.move_tuple_2_string[move]
                            
        self.expand[loc] = self.step
        self.step += 1


    def update_distances(self, iters_desired = 1):
        # take a node and expand it, counting iters
        
        iters_so_far = 0
        while iters_so_far < iters_desired:
            if len(self.locs_to_update) != 0:
                _, loc = self.locs_to_update.pop(0)
                self.expand_loc(loc)

            else:
                return "no more locations to visit"

            iters_so_far += 1
          
        if self.distances[self.end] != -1:
            self.expand_loc(self.end)
            return "problem solved!"
        elif len(self.locs_to_update) > 0:
            return "more locations available to visit"

        
    def iter_until_solved(self):
        status = "more locations available to visit"

        while status == "more locations available to visit" and self.distances[self.end] == -1:
            status = self.update_distances(1)
            
        return status
    
    
    def produce_path(self, loc):
        current_loc = loc
        path_graph = np.full(self.grid.shape, ' ')
        path_graph[loc] = '*'
        
        while current_loc != self.start:
            prev_move_str = self.moves_to_get_to_loc[current_loc]
            prev_move_tuple = self.move_string_2_tuple[prev_move_str]
            rev_move_tuple = (-prev_move_tuple[0], -prev_move_tuple[1])            
            
            current_loc = add_2tuple(current_loc, rev_move_tuple)
            path_graph[current_loc] = prev_move_str
        
        return path_graph


## unittests
import unittest

class map_tests(unittest.TestCase):
    
    def setUp(self):
        self.grid = np.array([[0, 0, 1, 0, 0, 0],
                              [0, 0, 1, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0],
                              [0, 0, 1, 1, 1, 0],
                              [0, 0, 0, 0, 1, 0]])
        self.init = [0, 0]
        self.goal = [len(grid)-1, len(grid[0])-1]
        self.cost = 1
        
        self.new_map = Map(self.grid, self.init, self.goal, self.cost)
    
    def test_map_initialization(self):
        self.assertTrue(np.array_equal(self.new_map.grid, self.grid))
        self.assertEqual(self.new_map.dims, (5, 6))
        
    def test_valid_and_legal_moves(self):
        # on grid tests
        self.assertTrue(self.new_map.valid_and_legal_move((4,3)))
        self.assertFalse(self.new_map.valid_and_legal_move((3,2)))
        
        # off grid checks
        self.assertFalse(self.new_map.valid_and_legal_move((-1,3)))
        self.assertFalse(self.new_map.valid_and_legal_move((6,3)))
        self.assertFalse(self.new_map.valid_and_legal_move((4,11)))
        self.assertFalse(self.new_map.valid_and_legal_move((4,-2)))
        
    def test_update_distances(self):
        self.new_map.update_distances(4)
        expected_distances = np.array([[ 0,  1, -1, -1, -1, -1],
                                       [ 1,  2, -1, -1, -1, -1],
                                       [ 2,  3, -1, -1, -1, -1],
                                       [-1, -1, -1, -1, -1, -1],
                                       [-1, -1, -1, -1, -1, -1]])
        self.assertTrue(np.array_equal(self.new_map.distances, expected_distances))
        
        response = self.new_map.update_distances(3)
        expected_distances = np.array([[ 0,  1, -1, -1, -1, -1],
                                       [ 1,  2, -1, -1, -1, -1],
                                       [ 2,  3,  4,  5, -1, -1],
                                       [ 3,  4, -1, -1, -1, -1],
                                       [-1, -1, -1, -1, -1, -1]])       
        self.assertEqual(response, "more locations available to visit")
        self.assertTrue(np.array_equal(self.new_map.distances, expected_distances))
        
        response = self.new_map.update_distances(15)
        self.assertEqual(response, "problem solved!")
        
    
    def test_iter_until_solved(self):
        response = self.new_map.iter_until_solved()
        self.assertEqual(response, "problem solved!")
        
        expected_distances = np.array([[ 0,  1, -1,  7,  8,  9],
                                       [ 1,  2, -1,  6,  7,  8],
                                       [ 2,  3,  4,  5, -1,  9],
                                       [ 3,  4, -1, -1, -1, 10],
                                       [ 4,  5,  6,  7, -1, 11]])
        self.assertTrue(np.array_equal(self.new_map.distances, expected_distances))
        
        expected_expand = np.array([[ 0,  1, -1, -1, -1, -1],
                                    [ 2,  3, -1, 14, 15, 16],
                                    [ 4,  5,  6,  7, -1, 17],
                                    [ 8,  9, -1, -1, -1, 18],
                                    [10, 11, 12, 13, -1, 19]])
        
        self.assertTrue(np.array_equal(self.new_map.expand, expected_expand))
        
    
    def test_produce_path(self):
        self.new_map.iter_until_solved()
        
        expected_path = np.array([[ '>', 'v', ' ', ' ', ' ', ' '],
                                  [ ' ', 'v', ' ', '>', '>', 'v'],
                                  [ ' ', '>', '>', '^', ' ', 'v'],
                                  [ ' ', ' ', ' ', ' ', ' ', 'v'],
                                  [ ' ', ' ', ' ', ' ', ' ', '*']])
        self.assertTrue(np.array_equal(self.new_map.produce_path(self.new_map.end), expected_path))
        
        expected_path = np.array([[ '>', 'v', ' ', ' ', ' ', ' '],
                                  [ ' ', 'v', ' ', '*', ' ', ' '],
                                  [ ' ', '>', '>', '^', ' ', ' '],
                                  [ ' ', ' ', ' ', ' ', ' ', ' '],
                                  [ ' ', ' ', ' ', ' ', ' ', ' ']])
        self.assertTrue(np.array_equal(self.new_map.produce_path((1,3)), expected_path))


if __name__ == "__main__":
    unittest.main(argv = ['first-arg-is-ignored'], verbosity=2, exit=False)

test_iter_until_solved (__main__.map_tests) ... ok
test_map_initialization (__main__.map_tests) ... ok
test_produce_path (__main__.map_tests) ... ok
test_update_distances (__main__.map_tests) ... ok
test_valid_and_legal_moves (__main__.map_tests) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.009s

OK
